In [ ]:
import os
from google.cloud import bigquery
import pandas as pd
from typing import List, Dict, Any

# --- Configuration ---
# NOTE: This script assumes you have authenticated with Google Cloud.
# If running locally, you can typically run 'gcloud auth application-default login'
# in your terminal first.

PROJECT_ID = "celtic-fact-367202"
DATASET_ID = "test"
SOURCE_TABLE_ID = "pokemon"
DESTINATION_TABLE_ID = "new_pokemon_data" # New table to demonstrate the push/upload function

def fetch_data(client: bigquery.Client, table_id: str) -> pd.DataFrame:
    """
    Fetches all data from the specified BigQuery table into a Pandas DataFrame.
    """
    full_table_path = f"{PROJECT_ID}.{DATASET_ID}.{table_id}"
    print(f"Fetching data from: {full_table_path}...")

    # A simple SQL query to select all data
    sql_query = f"""
        SELECT *
        FROM `{full_table_path}`
        LIMIT 100  -- Limiting the fetch for demonstration purposes
    """

    try:
        # The to_dataframe() method handles the job execution and result fetching
        df = client.query(sql_query).to_dataframe()
        print(f"Successfully fetched {len(df)} rows.")
        return df
    except Exception as e:
        print(f"An error occurred during data fetch: {e}")
        return pd.DataFrame()


def generate_mock_data() -> pd.DataFrame:
    """
    Generates a mock DataFrame that conforms to the 'pokemon' table schema
    for the purpose of demonstrating the 'push' operation.
    """
    print("Generating mock data for upload...")
    new_data = [
        {'#': 1001, 'Name': 'Pikachu Prime', 'Type 1': 'Electric', 'Type 2': None, 'HP': 150, 'Attack': 120, 'Defense': 90, 'Sp_Atk': 150, 'Sp_Def': 90, 'Speed': 180, 'Generation': 10, 'Legendary': True},
        {'#': 1002, 'Name': 'Bulbasaur Beta', 'Type 1': 'Grass', 'Type 2': 'Poison', 'HP': 100, 'Attack': 70, 'Defense': 70, 'Sp_Atk': 80, 'Sp_Def': 100, 'Speed': 60, 'Generation': 10, 'Legendary': False},
    ]

    # The columns MUST match the schema of the target BigQuery table
    columns = ['#', 'Name', 'Type 1', 'Type 2', 'HP', 'Attack', 'Defense', 'Sp_Atk', 'Sp_Def', 'Speed', 'Generation', 'Legendary']
    df = pd.DataFrame(new_data, columns=columns)
    return df


def push_data(client: bigquery.Client, df: pd.DataFrame, table_id: str):
    """
    Pushes data from a Pandas DataFrame to a specified BigQuery table.
    It will create the table if it does not exist.
    """
    full_table_path = f"{PROJECT_ID}.{DATASET_ID}.{table_id}"
    print(f"\nAttempting to push {len(df)} rows to: {full_table_path}...")

    # Configure the job
    job_config = bigquery.LoadJobConfig(
        # Set write disposition:
        # WRITE_TRUNCATE: Overwrites the table.
        # WRITE_APPEND: Appends to the table.
        # WRITE_EMPTY: Fails if the table is not empty.
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
    )

    try:
        # Start the load job
        job = client.load_table_from_dataframe(
            df, full_table_path, job_config=job_config
        )
        job.result()  # Wait for the job to complete

        table = client.get_table(full_table_path)
        print(f"Successfully loaded {table.num_rows} rows into {full_table_path}.")
    except Exception as e:
        print(f"An error occurred during data push: {e}")


def main():
    """Main function to run the BigQuery operations."""
    try:
        # Initialize the BigQuery client
        client = bigquery.Client()
    except Exception as e:
        print("Failed to initialize BigQuery client.")
        print("Please ensure your Google Cloud environment is authenticated.")
        print(f"Error: {e}")
        return

    # 1. GET DATA
    pokemon_df = fetch_data(client, SOURCE_TABLE_ID)

    if not pokemon_df.empty:
        print("\n--- Sample of Fetched Data ---")
        print(pokemon_df.head())
    else:
        print("Could not proceed without fetching data.")
        return

    # 2. PUSH DATA
    # Create the new data to push
    new_pokemon_data_df = generate_mock_data()

    # Upload the new data to the destination table
    push_data(client, new_pokemon_data_df, DESTINATION_TABLE_ID)

if __name__ == "__main__":
    main()
